<a href="https://colab.research.google.com/github/Abishethvarman/FakeNewsDetection/blob/datasets/V1.5.2Sentiment_Analysis_VADER%2BBoW_FakeNews_20k_ds_t0_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##VADER (Valence Aware Dictionary and sEntiment Reasoner) - Bag of words approach

Sentiment Analysis on referencing this https://www.kaggle.com/code/robikscube/sentiment-analysis-python-youtube-tutorial/notebook

DS used here is https://drive.google.com/file/d/1vdscq2CYrj2Gp4lH-JTplHGUq7lrJ6ts/view?usp=drive_link



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')

from PIL import Image
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

import nltk
import gensim
from nltk.corpus import stopwords
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.sentiment import SentimentIntensityAnalyzer
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('vader_lexicon')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

import plotly.express as px
from tqdm.notebook import tqdm

plt.style.use('ggplot')

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

In [3]:
%cd /mydrive/dataset/

/content/gdrive/My Drive/dataset


In [24]:
df=pd.read_csv("fake-20000-SA.csv")

In [25]:
print(df.shape)

(20000, 7)


In [26]:
df['original'] = df['title'] + ' ' + df['text']
df.head()

,Unnamed: 0,title,text,subject,date,isfake,label,original
0,10650,Trump Supporters Terrorize An American Woman ...,Donald Trump s deplorable supporters strike ag...,News,"December 3, 2016",1,fake,Trump Supporters Terrorize An American Woman ...
1,2041,Trump adviser's Russia credentials come under ...,MOSCOW (Reuters) - When Donald Trump named a l...,politicsNews,"August 24, 2016",0,real,Trump adviser's Russia credentials come under ...
2,8668,Turkey feels betrayed over EU accession but st...,LONDON (Reuters) - Turkey feels betrayed by so...,worldnews,"September 14, 2017",0,real,Turkey feels betrayed over EU accession but st...
3,1114,China newspapers say call with Taiwan's Tsai s...,SHANGHAI (Reuters) - Chinese state media on Mo...,politicsNews,"December 5, 2016",0,real,China newspapers say call with Taiwan's Tsai s...
4,13902,Sean Hannity Just Openly Threatened Someone O...,Is there some reason people like Donald Trump ...,News,"April 30, 2017",1,fake,Sean Hannity Just Openly Threatened Someone O...


In [30]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'one', 'two', 'more','said','u','however','still','get','many','could','u'])

In [27]:
# Preprocessing
def preprocess(text):
    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words and word.lower() != 'mor']
    return ' '.join(filtered_words)

In [31]:
# Apply preprocessing
df['clean'] = df['original'].apply(preprocess)

In [32]:
sia = SentimentIntensityAnalyzer()

In [33]:
df['id'] = df.index + 1

In [34]:
# Run the polarity score on the entire dataset
res = {}
for i, row in tqdm(df.iterrows(), total=len(df)):
    text = row['clean']
    myid = row['id']
    res[myid] = sia.polarity_scores(text)

  0%|          | 0/20000 [00:00<?, ?it/s]

In [35]:
vaders = pd.DataFrame(res).T
vaders = vaders.reset_index().rename(columns={'index': 'id'})
vaders = vaders.merge(df, how='left')
vaders.head(2)

,id,neg,neu,pos,compound,Unnamed: 0,title,text,subject,date,isfake,label,original,clean
0,1,0.294,0.572,0.134,-0.9961,10650,Trump Supporters Terrorize An American Woman ...,Donald Trump s deplorable supporters strike ag...,News,"December 3, 2016",1,fake,Trump Supporters Terrorize An American Woman ...,Trump Supporters Terrorize American Woman Subw...
1,2,0.054,0.848,0.098,0.9797,2041,Trump adviser's Russia credentials come under ...,MOSCOW (Reuters) - When Donald Trump named a l...,politicsNews,"August 24, 2016",0,real,Trump adviser's Russia credentials come under ...,Trump adviser 's Russia credentials come scrut...


In [36]:
# Convert VADER scores to binary sentiment labels
vaders['sentiment'] = vaders['compound'].apply(lambda x: 1 if x >= 0 else 0)

In [37]:
vaders.to_csv("fake-20000-SA-VADER-selected.csv")